In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys
import os
import glob
import shutil

import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(1, './modules/')
import pydelfi_t1.priors as priors
from smt.sampling_methods import LHS

#importing simulation modules and parameters file______
import simulator as sims
data_generation=sims.simulations()
from set_params import *

data_generation=sims.simulations()
#______________________________________________________
    
%matplotlib inline

import tensorflow as tf
print("TensorFlow {}\nnumpy {}".format(
    tf.__version__, np.__version__))

In [ ]:
checking_data = True

def check_data(data):
    print(data.shape)
    for j in range (n_obs):
        for i in range (int(100)):
            plt.hist(data[i,100*j:100*(j+1)])
        plt.show()

## Simulating real data samples

In [ ]:
n_sets = 100

In [ ]:
new_realdata = True

if new_realdata == True:
    if not os.path.exists('{}'.format(pathtorealsamples)):
        os.makedirs('{}'.format(pathtorealsamples))
    print('APPLYING LATIN HYPERCUBE')

    sampling = LHS(xlimits=np.array([[65,75],[-0.7,-0.3]]))
    draws = sampling(n_sets)
    
    print(np.corrcoef(draws[:,0],draws[:,1]))
    plt.scatter(draws[:,0],draws[:,1])
    plt.show()
    
    real_data = np.zeros((n_sets,(n_obs)*n_sources))
    
    for i in range (n_sets):
        theta=np.array([draws[i][0],draws[i][1]])
        sim = data_generation.data_simulator(theta = theta)
        real_data[i] = sim    

    np.savetxt('{}'.format(pathtorealsamples)+'/multi_thetas{}.txt'.format(ref_filename),draws)    
    np.savetxt('{}'.format(pathtorealsamples)+'/multi_randomdata{}.txt'.format(ref_filename),real_data)
    

if checking_data == True:
    data = np.loadtxt('{}'.format(pathtorealsamples)+'/multi_randomdata{}.txt'.format(ref_filename))
    check_data(data)

## Simulating training data for the regression neural network

In [ ]:
new_NN_data = True

if new_NN_data == True:
    data_generation.NN_datasets()
    
if checking_data == True:
    train_data = np.load('{}'.format(pathtoNNsims)+'training_datasets{}.npy'.format(ref_filename))
    check_data(train_data)
    val_data = np.load('{}'.format(pathtoNNsims)+'validation_datasets{}.npy'.format(ref_filename))
    check_data(val_data)

## Simulating the prerun simulation for LFI computations 

In [ ]:
new_prerunsims = True
n_sims = 2000

if new_prerunsims == True:
    
    pathlib.Path('{}'.format(pathtopydelfiprerunsims)).mkdir(parents=True, exist_ok=True)

    # drawing thetas

    print('APPLYING UNIFORM PRIOR')
    prior_distr = priors.Uniform(lower, upper)     

    draws = np.zeros((n_sims,2))
    for i in range (n_sims):
        draws[i]=prior_distr.draw()
    
    np.savetxt('{}'.format(pathtopydelfiprerunsims)+'/prerun_thetas{}.txt'.format(ref_filename),draws) 

    #sims

    prerun_sims = np.zeros((n_sims,(n_obs)*n_sources))
    for i in range (n_sims):
        theta=np.array([draws[i][0],draws[i][1]])
        sim = data_generation.data_simulator(theta = theta)
        prerun_sims[i] = sim  
    
    np.savetxt('{}'.format(pathtopydelfiprerunsims)+'/prerun_sims{}.txt'.format(ref_filename),prerun_sims)

    print(prerun_sims.shape)
    print(draws.shape)
    
if checking_data == True:
    prerun_data = np.loadtxt('{}'.format(pathtopydelfiprerunsims)+'/prerun_sims{}.txt'.format(ref_filename))
    check_data(prerun_data)

## Regression Neural Network 

In [ ]:
%cd ./modules/

In [ ]:
if sys.version[0]==3:
    pathlib.Path('{}'.format(pathtoNNmodels)).mkdir(parents=True, exist_ok=True) 
else:
    if not os.path.exists('{}'.format(pathtoNNmodels)):
        os.makedirs('{}'.format(pathtoNNmodels))

if MPI_process == True:
    !mpiexec -np 12 python NN.py
else:
    %run NN.py

## STAN

In [ ]:
%cd ./modules/

In [ ]:
for k in range (1):
    np.save('real_data_element',k)
    %run STAN.py
os.remove("real_data_element.npy")

## Likelihood free inference

In [ ]:
%cd ./modules/

In [ ]:
if sys.version[0]==3:
    pathlib.Path('{}'.format(pathtoLFIresults)).mkdir(parents=True, exist_ok=True) 
else:
    if not os.path.exists('{}'.format(pathtoLFIresults)):
        os.makedirs('{}'.format(pathtoLFIresults))

In [ ]:
all_real_data = True
if all_real_data == True:
    for k in range (100):
        np.save('real_data_element',k)
        if MPI_process == True:
            !mpiexec -np 12 python LFI.py
        else:
            %run LFI.py
        dest = '../LFI_results/results_pydelfi_{}'.format(k)
        if os.path.isdir(dest) == True:
            for file in glob.glob('../results_pydelfi/*'):
                shutil.move(file, dest)
        else:
            os.rename('../results_pydelfi',dest)
    os.remove('real_data_element.npy')
else:
    if MPI_process == True:
        !mpiexec -np 1 python LFI.py
    else:
        %run LFI.py